---
comments: true
layout: notebook
title: Class 3- JWT Implimentation
description: Teamteach by Lindsay, Anika, Grace, Samhita
type: collab
courses: { csa: {week: 19} }
---

# JWT Implimentation

![JWT](https://files.catbox.moe/t69vm3.png)

## What is a JWT (JSON Web Token)?

- JSON Web Tokens (JWT) serve as a secure means of transmitting information in JSON format, providing a way to establish trust between different parties in a web application. 
- Typically used to transmit information about an authenticated user and additional metadata (usernames, user ID, email, roles, etc.).

### Parts of a JWT:

![Parts](https://files.catbox.moe/19taua.png)

- Each section is divided by a dot
- Header (red): Contains information about how the JWT is encoded, such as the type of token (JWT) and the signing algorithm being used.
- Payload (purple): Contains the claims. Claims are statements about an entity (typically, the user) and additional data. There are three types of claims: registered, public, and private claims.
- Signature (teal): Created by combining the encoded header, encoded payload, and a secret (or key) using the specified algorithm. This signature is used to verify that the sender of the JWT is who it says it is and to ensure that the message wasn't changed along the way.

### HMAC and RSA or ECDSA Keys:

- Used for signing JWT tokens
- HMAC Algorithm (Secret Key): 
  - HMAC (Hash-based Message Authentication Code)
  - A secret key is used to both create and verify the signature.
  - The issuer (who creates the JWT) uses a secret key and a hashing algorithm to create a signature by combining the encoded header and payload.
  - The recipient (who validates the JWT) uses the same secret key to independently calculate the signature and verify that it matches the one provided in the JWT.
  - Symmetric key solution: same key used for signing and verification
- RSA or ECDSA (Public/Private Key Pair):
  - RSA (Rivest–Shamir–Adleman) or ECDSA (Elliptic Curve Digital Signature Algorithm)
  - A pair of keys is used: a private key for signing and a corresponding public key for verification.
  - The issuer signs the JWT using their private key, and the signature is included in the JWT.
  - The recipient, who wants to verify the JWT, uses the public key associated with the issuer to check the signature's validity.
  - Asymmetric key solution: private key is kept secret, and only the public key is shared.

## Rest APIs and JSON Web Tokens

### Rest APIs:

- REST (Representational State Transfer) API (Application Programming Interface)
- Architectural style for designing networked applications
- Set of rules and conventions for building and interacting with web services that adhere to the principles of REST.

Principals of REST:
- Statelessness: Each request from a client to a server must contain all the information needed to understand and fulfill the request. The server should not store any information about the client's state between requests. This makes the system scalable and easy to maintain.
- Client-Server Architecture: The client and server are separate entities that communicate over a network. The client is responsible for the user interface and user experience, while the server is responsible for processing requests, managing resources, and handling business logic.
- Uniform Interface: RESTful APIs have a uniform and consistent interface, which simplifies the architecture and promotes a clear separation of concerns. The uniform interface is defined by several constraints, including resource identification through URIs (Uniform Resource Identifiers), resource manipulation through representations, and self-descriptive messages.
- Representation: Resources can have multiple representations, such as JSON or XML, which can be selected based on the client's needs or capabilities. The representation contains the current state of the resource.

### REST APIs and JWT:

- 

## Implimenting JWT in Flask

#### 1) Install Flas-JWT-Extended

In [ ]:
pip install Flask-JWT-Extended

#### 2) Include JWT Library in Flask App Setup

In [ ]:
from flask_jwt_extended import JWTManager

# You don't have to add this again if you already have it.
app = Flask(__name__)

# Setup the Flask-JWT-Extended extension
app.config["JWT_SECRET_KEY"] = "secret"  # Remember to change "secret" to a more complex key
jwt = JWTManager(app)

#### 3) Create an Endpoint for Generating Tokens

Using POST because it is creating tokens (POST is for creation)

In [ ]:
POST /token
Content-type: application/json
Body:
{
     "username": "alesanchezr",
     "password": "12341234"
}

In [ ]:
from flask_jwt_extended import create_access_token

# Create a route to authenticate your users and return JWT Token
# The create_access_token() function is used to actually generate the JWT
@app.route("/token", methods=["POST"])
def create_token():
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    # Query your database for username and password
    user = User.query.filter_by(username=username, password=password).first()

    if user is None:
        # The user was not found on the database
        return jsonify({"msg": "Bad username or password"}), 401
    
    # Create a new token with the user id inside
    access_token = create_access_token(identity=user.id)
    return jsonify({ "token": access_token, "user_id": user.id })

#### 4) Use @jwt_required() Decorator on Private Routes

Endpoints that require authorization (private endpoints) should use the @jwt_required() decorator. You can retrieve valid, authenticated user information using the get_jwt_identity function. 

In [ ]:
from flask_jwt_extended import jwt_required, get_jwt_identity

# Protect a route with jwt_required, which will kick out requests without a valid JWT
@app.route("/protected", methods=["GET"])
@jwt_required()
def protected():
    # Access the identity of the current user with get_jwt_identity
    current_user_id = get_jwt_identity()
    user = User.query.get(current_user_id)
    
    return jsonify({"id": user.id, "username": user.username }), 200

## Implimenting JWT in Front-End

#### 1) Create a New Token

Based on earlier endpoints, we have to POST /token with username + password information in request body.

In [ ]:
const login = async (username, password) => {
     const resp = await fetch(`https://your_api.com/token`, { 
          method: "POST",
          headers: { "Content-Type": "application/json" },
          body: JSON.stringify({ username, password }) 
     })

     if(!resp.ok) throw Error("There was a problem in the login request")

     if(resp.status === 401){
          throw("Invalid credentials")
     }
     else if(resp.status === 400){
          throw ("Invalid email or password format")
     }
     const data = await resp.json()
     // Save your token in the localStorage
     // Also you should set your user into the store using the setItem function
     localStorage.setItem("jwt-token", data.token);

     return data
}

#### 2) Retrieving Information

Now, lets say I've logged in to the front-end application and I want to retrieve sensitive data:

In [ ]:
// Assuming "/protected" is a private endpoint
const getMyTasks = async () => {
     // Retrieve token from localStorage
     const token = localStorage.getItem('jwt-token');

     const resp = await fetch(`https://your_api.com/protected`, {
        method: 'GET',
        headers: { 
          "Content-Type": "application/json",
          'Authorization': 'Bearer ' + token // ⬅⬅⬅ authorization token
        } 
     });

     if(!resp.ok) {
          throw Error("There was a problem in the login request")
     } else if(resp.status === 403) {
          throw Error("Missing or invalid token");
     } else {
         throw Error("Unknown error");
     }

     const data = await resp.json();
     console.log("This is the data you requested", data);
     return data
}